# Дообучение сегментатора на полных изображениях

In [24]:
from model import RootRegressor
from dataset import RootVolumeDataset
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import random
import os
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from glob import glob
from PIL import Image

In [17]:
def nearest_power_of_2(n):
    return 2 ** int(np.ceil(np.log2(n)))

def find_max_slice(csv_path):
    df = pd.read_csv(csv_path)
    return nearest_power_of_2(np.max(df["End"] - df["Start"]) + 1)

def get_params():
    size_file = open("target_size.txt", "r")
    width, height = [int(line.replace("\n", "").split(" ")[-1]) for line in size_file.readlines()]
    params = {}
    params["width"] = width
    params["height"] = height
    params["max_slices"] = find_max_slice("Train.csv")
    return params

In [18]:
params = get_params()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_path = "Train.csv"
img_path = "images/train/"
dataset = RootVolumeDataset(csv_path=train_path,
                            img_root=img_path,
                            target_width=params['width'],
                            target_height=params['height'],
                            label_root="train_labels/",
                            device=device,
                            pre_segment=False,
                            full_img=False)

In [19]:
print(dataset[0]["images"])

[<PIL.Image.Image image mode=RGB size=908x14 at 0x120BD751CF0>, <PIL.Image.Image image mode=RGB size=908x14 at 0x120BD779C60>, <PIL.Image.Image image mode=RGB size=908x14 at 0x120BD779FF0>, <PIL.Image.Image image mode=RGB size=908x14 at 0x120BD77A140>, <PIL.Image.Image image mode=RGB size=908x14 at 0x120FC6B5B40>, <PIL.Image.Image image mode=RGB size=908x14 at 0x120A17D75B0>, <PIL.Image.Image image mode=RGB size=908x14 at 0x120A17D7A60>, <PIL.Image.Image image mode=RGB size=908x14 at 0x120A17D40D0>, <PIL.Image.Image image mode=RGB size=908x14 at 0x120A17D6CB0>, <PIL.Image.Image image mode=RGB size=908x14 at 0x120A17D4AC0>]


In [20]:
imgs = dataset[23]["images"]
for image in imgs:
    display(image)

In [25]:
def stack_images_vertically(images : list[Image]):
    widths, heights = zip(*(i.size for i in images))
    min_width = min(widths)
    total_height = sum(heights)
    stacked_image = Image.new("RGB", (min_width, total_height))
    y_offset = 0
    for img in images:
        if img.width > min_width:
            img = img.crop((0, 0, min_width, img.height))
        stacked_image.paste(img,(0, y_offset))
        y_offset += img.height
    return stacked_image

In [26]:
def generate_random_color():
    """Generate a bright, random color."""
    return tuple(random.randint(100, 255) for _ in range(3))  # RGB values

def prepare_stacked_data(image_paths, label_paths):
    """
    Prepare a stacked image and combined mask string from matching image-label pairs.

    Args:
        image_paths (list): List of paths to image files.
        label_paths (list): List of paths to label files.

    Returns:
        tuple: (stacked_image, combined_masks_str)
            - stacked_image: PIL Image object of the stacked images.
            - combined_masks_str: String combining all mask annotations.
    """
    # Create a dictionary mapping label base names to their full paths
    label_map = {os.path.splitext(os.path.basename(lf))[0]: lf for lf in label_paths}
    matching_images = []
    matching_labels = []

    # Match images with corresponding labels
    for image_path in image_paths:
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        if image_name in label_map:
            matching_images.append(image_path)
            matching_labels.append(label_map[image_name])


    # Load and stack the matching images
    images = [Image.open(path) for path in matching_images]
    if len(images) == 0:
        images = [Image.open(path) for path in image_paths]
    widths, heights = zip(*(i.size for i in images))
    min_width = min(widths)
    total_height = sum(heights)

    # Stack images vertically
    stacked_image = Image.new('RGB', (min_width, total_height))
    y_offset = 0
    for im in images:
        im_resized = im.resize((min_width, im.height))
        stacked_image.paste(im_resized, (0, y_offset))
        y_offset += im.height

    # Handle case where no matching pairs are found
    if not matching_images:
        print("No matching image-label pairs found. Just return the image")
        return stacked_image, ""

    # Combine masks with adjusted coordinates
    combined_mask_lines = []
    cumulative_height = 0
    for label_path, im_height, im_width in zip(matching_labels, heights, widths):
        with open(label_path, 'r') as file:
            for line in file:
                parts = line.strip().split()
                if len(parts) < 3:
                    continue
                class_id = parts[0]
                coords = list(map(float, parts[1:]))
                new_coords = []
                for x_norm, y_norm in zip(coords[0::2], coords[1::2]):
                    x_pixel = int(x_norm * im_width)
                    y_pixel = int(y_norm * im_height)
                    x_pixel_cropped = min(x_pixel, min_width - 1)
                    y_pixel_offset = y_pixel + cumulative_height
                    new_x_norm = x_pixel_cropped / min_width
                    new_y_norm = y_pixel_offset / total_height
                    new_coords.extend([new_x_norm, new_y_norm])
                new_line = class_id + " " + " ".join(f"{val:.6f}" for val in new_coords)
                combined_mask_lines.append(new_line)
        cumulative_height += im_height
    combined_masks_str = "\n".join(combined_mask_lines)

    return stacked_image, combined_masks_str

def get_polygons_from_mask_str(mask_str, img_width, img_height):
    polygons = []
    # Split into lines (each line is a polygon)
    lines = mask_str.strip().split('\n') 
    for line in lines:
        # Split line into parts (class ID and coordinates)
        parts = line.strip().split()
        if len(parts) < 3:  # Need at least class ID + one (x, y) pair
            continue
        # Convert coordinates to floats, skipping the class ID
        coords = list(map(float, parts[1:]))
        if len(coords) % 2 != 0:  # Must have even number for (x, y) pairs
            continue
        # Group into (x, y) pairs and convert to pixel coordinates
        points = [(int(x * img_width), int(y * img_height)) 
                  for x, y in zip(coords[0::2], coords[1::2])]
        # Convert to NumPy array and append to list
        polygons.append(np.array(points, dtype=np.int32))
    return polygons
def visualize(stacked_image, polygons):
    # Visualize with polygons
    image = np.array(stacked_image)
    for polygon in polygons:
        if polygon.shape[0]:
            color = generate_random_color()
            cv2.polylines(image, [polygon.astype(int)], isClosed=True, color=color, thickness=1)
    plt.imshow(image)
    plt.axis('off')
    plt.show()


# Обучение

In [27]:
train = pd.read_csv("Train.csv")
output = "seg/images"
os.makedirs(output,exist_ok=True)
os.makedirs(output.replace('images', 'labels'), exist_ok=True)

In [28]:
def get_images_labels(folder_name):
    """Retrieve sorted lists of label and image file paths for a given folder."""
    labels = glob(f'train_labels/{folder_name}*')
    labels = sorted(labels)
    imgs = glob(f'images/train/{folder_name.split("_")[0]}/{folder_name}*')
    imgs = sorted(imgs)
    return labels, imgs

## Подготовим изображения и маски

In [29]:
for f in tqdm(train.FolderName.unique()):
    for side in ['L', 'R']:
        labels, imgs = get_images_labels(f'{f}_{side}_')
        if imgs and labels:
            stacked_image, mask_str = prepare_stacked_data(imgs, labels)
            output_path = f'{output}/{f}_{side}.png'
            stacked_image.save(output_path)
            with open(output_path.replace('images', 'labels').replace('.png', '.txt'), 'w') as fx:
                fx.write(mask_str)


100%|██████████| 98/98 [00:08<00:00, 11.46it/s]
